In [ ]:


from rdflib import Graph, Namespace, RDF, RDFS, Literal, BNode, SH
from pyshacl import validate

# 1) Load data and shapes
data_graph = Graph().parse(
    "data/rdf/epd_rdf_instance_datastore_canonical_skos_din_bki.ttl",
    format="turtle",
)
shapes_graph = Graph().parse("data/rdf/concrete_class_shacl_shapes.ttl", format="turtle")

# 2) Run pySHACL (advanced features, rule iteration, and inplace inference)
conforms, results_graph, results_text = validate(
    data_graph,
    shacl_graph=shapes_graph,
    advanced=True,
    iterate_rules=True,
    inference="rdfs",
    debug=True,
    inplace=True,  # inferred triples are added directly into data_graph
)

print("Conforms?", conforms)
print("Results:\n", results_text)

# Merge any separately returned inferred triples (if any) into data_graph
data_graph += results_graph

# 3) Clean-up unwanted triples with a single loop.
# Define any polluting subjects you want to remove.
polluting_subjects = {RDF.type, RDFS.subPropertyOf}
polluting_predicate = {SH.conforms, RDFS.subPropertyOf}
polluting_objects = {RDF.Property, RDFS.Resource}
# Also, set up the SHACL namespace for later comparisons.
SHACL = Namespace("http://www.w3.org/ns/shacl#")

for s, p, o in list(data_graph.triples((None, None, None))):
    if (isinstance(s, Literal)
        or (isinstance(s, BNode) and o == SHACL.ValidationReport)
        or (s in polluting_subjects)
        or (p in polluting_predicate)
        or (o in polluting_objects)
    ):
        data_graph.remove((s, p, o))

# 4) Bind preferred prefix for serialization.
CC = Namespace("https://example.org/concreteclass/")
data_graph.bind("cc", CC)

# 5) Serialize the cleaned-up graph.
output_path = "data/rdf/epd_rdf_instance_datastore_canonical_skos_din_bki_shacl.ttl"
data_graph.serialize(output_path, format="turtle")
print(f"Cleaned graph serialized to {output_path}")


In [ ]:
# RDFlib

from rdflib import Graph, Namespace, RDF, Literal, XSD
from rdflib.namespace import SKOS

# Namespaces
CC = Namespace("https://example.org/concreteclass/")
ILCD = Namespace("https://example.org/ilcd/")

# Create graph and bind prefixes
g = Graph()
g.bind("cc", CC)
g.bind("skos", SKOS)
g.bind("ilcd", ILCD)

# 1) Define the SKOS concepts (Strength)
g.add((CC.LowStrengthConcrete, RDF.type, SKOS.Concept))
g.add((CC.LowStrengthConcrete, SKOS.prefLabel, Literal("Low Strength Concrete", "en")))
g.add(
    (
        CC.LowStrengthConcrete,
        SKOS.note,
        Literal("Compressive Strength ≤ 25 MPa (e.g., ≤ C16/20)", "en"),
    )
)

g.add((CC.MediumStrengthConcrete, RDF.type, SKOS.Concept))
g.add(
    (
        CC.MediumStrengthConcrete,
        SKOS.prefLabel,
        Literal("Medium Strength Concrete", "en"),
    )
)
g.add(
    (
        CC.MediumStrengthConcrete,
        SKOS.note,
        Literal("Compressive Strength 25–40 MPa (e.g., C20/25–C30/37)", "en"),
    )
)

g.add((CC.HighStrengthConcrete, RDF.type, SKOS.Concept))
g.add(
    (CC.HighStrengthConcrete, SKOS.prefLabel, Literal("High Strength Concrete", "en"))
)
g.add(
    (
        CC.HighStrengthConcrete,
        SKOS.note,
        Literal("Compressive Strength ≥ 40 MPa (e.g., ≥ C35/45)", "en"),
    )
)

# 2) Define the SKOS concepts (Weight)
g.add((CC.LightWeightConcrete, RDF.type, SKOS.Concept))
g.add((CC.LightWeightConcrete, SKOS.prefLabel, Literal("Light Weight Concrete", "en")))
g.add((CC.LightWeightConcrete, SKOS.note, Literal("Density 800–2000 kg/m³", "en")))

g.add((CC.NormalWeightConcrete, RDF.type, SKOS.Concept))
g.add(
    (CC.NormalWeightConcrete, SKOS.prefLabel, Literal("Normal Weight Concrete", "en"))
)
g.add((CC.NormalWeightConcrete, SKOS.note, Literal("Density 2000–2600 kg/m³", "en")))

g.add((CC.HeavyWeightConcrete, RDF.type, SKOS.Concept))
g.add((CC.HeavyWeightConcrete, SKOS.prefLabel, Literal("Heavy Weight Concrete", "en")))
g.add((CC.HeavyWeightConcrete, SKOS.note, Literal("Density > 2600 kg/m³", "en")))

# 3) Parse existing EPD data
g.parse(
    "data/rdf/epd_rdf_instance_datastore_canonical_skos_din_bki.ttl", format="turtle"
)

# 4) Classify by compressive strength
strength_query = """
PREFIX ilcd: <https://example.org/ilcd/>
SELECT ?epd ?val
WHERE {
  ?epd a ilcd:ProcessDataSet ;
       ilcd:exchanges ?exchanges .
  ?exchanges ilcd:exchange ?exchangeEntry .
  ?exchangeEntry ilcd:materialProperties ?mp .
  ?mp ilcd:name "compressive strength" ;
      ilcd:value ?strVal .
  BIND(xsd:float(?strVal) AS ?val)
}
"""
strength_results = g.query(strength_query)

for row in strength_results:
    epd_uri = row["epd"]
    cs_val = float(row["val"])

    if cs_val < 25:
        concept = CC.LowStrengthConcrete
    elif cs_val <= 40:
        concept = CC.MediumStrengthConcrete
    else:
        concept = CC.HighStrengthConcrete

    g.add((epd_uri, CC.hasStrengthClassification, concept))

# 5) Classify by bulk density
density_query = """
PREFIX ilcd: <https://example.org/ilcd/>
SELECT ?epd ?val
WHERE {
  ?epd a ilcd:ProcessDataSet ;
       ilcd:exchanges ?exchanges .
  ?exchanges ilcd:exchange ?exchangeEntry .
  ?exchangeEntry ilcd:materialProperties ?mp .
  ?mp ilcd:name "gross density" ;
      ilcd:value ?denVal .
  BIND(xsd:float(?denVal) AS ?val)
}
"""
density_results = g.query(density_query)

for row in density_results:
    epd_uri = row["epd"]
    bd_val = float(row["val"])

    if bd_val < 2000:
        concept = CC.LightWeightConcrete
    elif bd_val <= 2600:
        concept = CC.NormalWeightConcrete
    else:
        concept = CC.HeavyWeightConcrete

    g.add((epd_uri, CC.hasWeightClassification, concept))

# 6) Serialize final updated graph
g.serialize(
    "data/rdf/epd_rdf_instance_datastore_canonical_skos_din_bki_concrete.ttl",
    format="turtle",
)
print(
    "Classification complete. Output saved to epd_rdf_instance_datastore_canonical_skos_din_bki_concrete.ttl"
)
